<font size="3" color="yellow">  
Construct the Jacobian matrix for the 6DoF manipulator.
</font>

In [1]:
import numpy as np
import math
import sympy as sp

In [2]:
def symht(a, alpha, d, q):
    """
    Calculate Homegeneous Transformation Matrix of nth frame w.r.t. base(0th frame)
    """
    c_q = sp.cos(q)
    s_q = sp.sin(q)
    c_alpha = sp.cos(alpha)
    s_alpha = sp.sin(alpha)

    T = sp.Matrix([
        [c_q, -s_q * c_alpha, s_q * s_alpha, a * c_q],
        [s_q, c_q * c_alpha, -c_q * s_alpha, a * s_q],
        [0, s_alpha, c_alpha, d],
        [0, 0, 0, 1]
    ])
    return T

<table>
    <tr>
        <td><img src="images/jacobian.png" width = "600" height = "300"/></td>
    </tr>
</table>

In [3]:
def cal_jacobian_mat(q) -> np.ndarray:
    """
    Compute the Jacobian Matrix
    
    :param q: list of joint angles (in rad)
    :type params: numpy.ndarray
    :return J: Jacobian Matrix
    :rtype: numpy.ndarray
    """
    # Define symbolic variables
    q1, q2, q3, q4, q5, q6 = sp.symbols('q1 q2 q3 q4 q5 q6')
    q_sym = [q1, q2, q3, q4, q5, q6]

    a = [0, 1, 0, 0, 0, 0]
    d = [1, 0, 0, 1, 0, 1]
    alpha = [sp.pi/2, 0, sp.pi/2, -sp.pi/2, sp.pi/2, 0]

    # Compute transformation matrices
    T00 = sp.eye(4)
    T01 = symht(q1, d[0], a[0], alpha[0])
    T12 = symht(q2, d[1], a[1], alpha[1])
    T23 = symht(q3 + sp.pi/2, d[2], a[2], alpha[2])
    T34 = symht(q4, d[3], a[3], alpha[3])
    T45 = symht(q5, d[4], a[4], alpha[4])
    T56 = symht(q6, d[5], a[5], alpha[5])

    T02 = T01 * T12
    T03 = T02 * T23
    T04 = T03 * T34
    T05 = T04 * T45
    T06 = T05 * T56

    # Extract z vectors for rotation axes
    z0 = T00[0:3, 2]
    z1 = T01[0:3, 2]
    z2 = T02[0:3, 2]
    z3 = T03[0:3, 2]
    z4 = T04[0:3, 2]
    z5 = T05[0:3, 2]

    # Compute position vectors
    p06 = T06[0:3, 3]

    # Compute Jacobian columns
    j1 = sp.Matrix.vstack(sp.diff(p06, q1), z0)
    j2 = sp.Matrix.vstack(sp.diff(p06, q2), z1)
    j3 = sp.Matrix.vstack(sp.diff(p06, q3), z2)
    j4 = sp.Matrix.vstack(sp.diff(p06, q4), z3)
    j5 = sp.Matrix.vstack(sp.diff(p06, q5), z4)
    j6 = sp.Matrix.vstack(sp.diff(p06, q6), z5)

    # Assemble Jacobian Matrix
    J = sp.Matrix.hstack(j1, j2, j3, j4, j5, j6)

    # Substitute numerical values of joint angles
    J_numeric = J.subs({q_sym[i]: q[i] for i in range(6)})

    # Convert symbolic Jacobian to numeric
    J_numeric = np.array(J_numeric).astype(np.float64)

    return J_numeric

In [4]:
q_values = [0.1, np.pi/2, 0.3, np.pi/2, 0.5, 0.6]
J = cal_jacobian_mat(q_values)
print(J)

[[ 0.          0.         -0.54030231  0.          0.41614684  0.41614684]
 [ 1.          1.          0.          1.          0.          0.        ]
 [ 0.          0.          0.84147098  0.          0.90929743  0.90929743]
 [ 0.          0.84147098  0.84147098  0.84147098  0.90929743  0.90929743]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 1.          0.54030231  0.54030231  0.54030231 -0.41614684 -0.41614684]]


<font size="3" color="yellow">
Forward velocity kinematics
</font>



(Given joint velocities, compute end-effector velocity, i.e., linear and angular velocities)

$\dot{x} = J\dot{q}$

In [5]:
qd = np.array([0.1, 0.2, -0.1, 0.3, 0.0, 0.2])

In [6]:
xd = J @ qd.T

In [7]:
xd

array([0.1372596 , 0.6       , 0.09771239, 0.51844788, 0.        ,
       0.23289156])

<font size="3" color="yellow">
Inverse velocity kinematics
</font>

(Given desired end-effector velocity, compute joint velocities)

$\dot{q} = J^{+}\dot{x}$, where $J^{+} = J^{T}(JJ^{T})^{-1}$ is the Moore-Penrose pseudoinverse of matrix J.

In [8]:
def inv_jacob(q):
    """
    Compute Inverse Jacobian Matrix.

    If the Jacobian Matrix is singular, then it computes Moore-Penrose pseudoinverse of matrix J in place of simple inverse of J 
    """
    J = cal_jacobian_mat(q)
    if J.shape[0] == J.shape[1] and np.linalg.det(J) != 0:
        J_inv = np.linalg.inv(J)
    else:
        print("The Jacobian Matrix is singular. Using Moore-Penrose pseudoinverse of matrix J!")
        J_inv = np.linalg.pinv(J)

    return J_inv

In [9]:
xd = np.array([0.1372596 , 0.6, 0.09771239, 0.51844788, 0, 0.23289156])

In [10]:
J_inv = inv_jacob(q_values)

The Jacobian Matrix is singular. Using Moore-Penrose pseudoinverse of matrix J!


In [11]:
J_inv

array([[ 0.63900887,  0.36099113,  0.83930297, -0.83930297,  0.        ,
         0.63900887],
       [-0.18049557,  0.18049557, -0.49559238,  0.49559238,  0.        ,
        -0.18049557],
       [-0.7050647 , -0.37553991,  0.28938848,  0.20515839,  0.        ,
         0.37553991],
       [-0.18049557,  0.18049557, -0.49559238,  0.49559238,  0.        ,
        -0.18049557],
       [ 0.40975222,  0.09024778,  0.3703489 , -0.04930259,  0.        ,
        -0.09024778],
       [ 0.40975222,  0.09024778,  0.3703489 , -0.04930259,  0.        ,
        -0.09024778]])

In [12]:
qd = J_inv @ xd.T

In [13]:
qd

array([ 0.10000001,  0.25      , -0.1       ,  0.25      ,  0.1       ,
        0.1       ])

In [5]:
# T_01 = cal_homo_trans_mat_0_to_n(1)
# T_02 = cal_homo_trans_mat_0_to_n(2)
# T_03 = cal_homo_trans_mat_0_to_n(3)
# T_04 = cal_homo_trans_mat_0_to_n(4)
# T_05 = cal_homo_trans_mat_0_to_n(5)
# T_06 = cal_homo_trans_mat_0_to_n(6)

In [6]:
# Z_00 = np.array([0, 0, 1])
# Z_01 = T_01[:3,:3] @ np.array([0, 0, 1])
# Z_02 = T_02[:3,:3] @ np.array([0, 0, 1])
# Z_03 = T_03[:3,:3] @ np.array([0, 0, 1])
# Z_04 = T_04[:3,:3] @ np.array([0, 0, 1])
# Z_05 = T_05[:3,:3] @ np.array([0, 0, 1])
# Z_06 = T_06[:3,:3] @ np.array([0, 0, 1])

# P0 = np.array([0, 0, 0])
# P1 = T_01[:3, 3]
# P2 = T_02[:3, 3]
# P3 = T_03[:3, 3]
# P4 = T_04[:3, 3]
# P5 = T_05[:3, 3]
# P6 = T_06[:3, 3]

In [ ]:
# J_v = np.array([np.cross(Z_00, P6 - P0), 
#                 np.cross(Z_01, P6 - P1), 
#                 np.cross(Z_02, P6 - P2), 
#                 np.cross(Z_03, P6 - P3), 
#                 np.cross(Z_04, P6 - P4), 
#                 np.cross(Z_05, P6 - P5)])
# J_w = np.array([Z_00, Z_01, Z_02, Z_03, Z_04, Z_05])
# J = np.vstack([np.column_stack(J_v), np.column_stack(J_w)])

In [ ]:
params = np.array([[0, a2, 0, 0, 0, 0], 
                  [np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0], 
                  [d1, 0, 0, d4, 0, d6], 
                  [theta[0], theta[1], theta[2] + np.pi/2, theta[3], theta[4], theta[5]]])

In [17]:
cal_jacobian_mat(np.array([0.1, 0.2, -0.1, 0.3, 0.0, 0.2]))

array([[ 0.        ,  0.        , -0.54030231,  0.        ,  0.41614684,
         0.41614684],
       [ 1.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.84147098,  0.        ,  0.90929743,
         0.90929743],
       [ 0.        ,  0.84147098,  0.84147098,  0.84147098,  0.90929743,
         0.90929743],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.        ,  0.54030231,  0.54030231,  0.54030231, -0.41614684,
        -0.41614684]])

In [10]:
type(qd)

numpy.ndarray

In [35]:
xd

array([-0.18751723, -0.07457943, -0.01298238, -0.12026682, -0.20893556,
        0.55427828])

<font size="3">
Inverse velocity kinematics
</font>

(Given desired end-effector velocity, compute joint velocities)

$\dot{q} = J^{+}\dot{x}$, where $J^{+} = J^{T}(JJ^{T})^{-1}$ is the Moore-Penrose pseudoinverse of matrix J.

In [ ]:
def inv_jacob(q):
    ''''''
    J = cal_jacobian_mat(q)
    if J.shape[0] == J.shape[1] and np.linalg.det(J) != 0:
        J_inv = np.linalg.inv(J)
    else:
        print("The Jacobian Matrix isn't singular. Using Moore-Penrose pseudoinverse of matrix J!")
        J_inv = np.linalg.pinv(J)

    return J_inv

In [72]:
det = np.linalg.det(J)
if np.isclose(det, 0):
    print("The matrix is singular.")
else:
    print("The matrix is not singular.")

The matrix is singular.


In [73]:
qd = np.linalg.pinv(J) @ xd

In [74]:
qd

array([ 1.00000000e-01,  2.00000000e-01, -1.00000000e-01,  2.50000000e-01,
       -6.10622664e-16,  2.50000000e-01])

In [ ]:
# J = np.block([[np.cross(Z_00, P6 - P0), Z_00], [np.cross(Z_01, P6 - P1), Z_01], [np.cross(Z_02, P6 - P2), Z_02], [np.cross(Z_03, P6 - P3), Z_03], [np.cross(Z_04, P6 - P4), Z_04], [np.cross(Z_05, P6 - P5), Z_05]])

In [5]:
import sympy as sp
import numpy as np

In [2]:
def symht(q, d, a, alpha):
    c_q = sp.cos(q)
    s_q = sp.sin(q)
    c_alpha = sp.cos(alpha)
    s_alpha = sp.sin(alpha)

    T = sp.Matrix([
        [c_q, -s_q * c_alpha, s_q * s_alpha, a * c_q],
        [s_q, c_q * c_alpha, -c_q * s_alpha, a * s_q],
        [0, s_alpha, c_alpha, d],
        [0, 0, 0, 1]
    ])
    return T

In [7]:
def jacob(q):
    # Define symbolic variables
    q1, q2, q3, q4, q5, q6 = sp.symbols('q1 q2 q3 q4 q5 q6')
    q_sym = [q1, q2, q3, q4, q5, q6]

    a = [0, 1, 0, 0, 0, 0]
    d = [1, 0, 0, 1, 0, 1]
    alpha = [sp.pi/2, 0, sp.pi/2, -sp.pi/2, sp.pi/2, 0]

    # Compute transformation matrices
    T00 = sp.eye(4)
    T01 = symht(q1, d[0], a[0], alpha[0])
    T12 = symht(q2, d[1], a[1], alpha[1])
    T23 = symht(q3, d[2], a[2], alpha[2])
    T34 = symht(q4, d[3], a[3], alpha[3])
    T45 = symht(q5, d[4], a[4], alpha[4])
    T56 = symht(q6, d[5], a[5], alpha[5])

    T02 = T01 * T12
    T03 = T02 * T23
    T04 = T03 * T34
    T05 = T04 * T45
    T06 = T05 * T56

    # Extract z vectors for rotation axes
    z0 = T00[0:3, 2]
    z1 = T01[0:3, 2]
    z2 = T02[0:3, 2]
    z3 = T03[0:3, 2]
    z4 = T04[0:3, 2]
    z5 = T05[0:3, 2]

    # Compute position vectors
    p06 = T06[0:3, 3]

    # Compute Jacobian columns
    j1 = sp.Matrix.vstack(sp.diff(p06, q1), z0)
    j2 = sp.Matrix.vstack(sp.diff(p06, q2), z1)
    j3 = sp.Matrix.vstack(sp.diff(p06, q3), z2)
    j4 = sp.Matrix.vstack(sp.diff(p06, q4), z3)
    j5 = sp.Matrix.vstack(sp.diff(p06, q5), z4)
    j6 = sp.Matrix.vstack(sp.diff(p06, q6), z5)

    # Assemble Jacobian Matrix
    J = sp.Matrix.hstack(j1, j2, j3, j4, j5, j6)

    # Substitute numerical values of joint angles
    J_numeric = J.subs({q_sym[i]: q[i] for i in range(6)})

    # Convert symbolic Jacobian to numeric
    J_numeric = np.array(J_numeric).astype(np.float64)

    return J_numeric

In [10]:
q_values = [0.1, np.pi/2, 0.3, np.pi/2, 0.5, 0.6]
J = jacob(q_values)
print(J)

[[ 0.2979569  -1.54709575 -0.55209158  0.14097212 -0.36811249  0.        ]
 [ 1.83262468 -0.15522734 -0.05539393  0.01414439 -0.91892328  0.        ]
 [ 0.          1.79372313  1.79372313 -0.45801271 -0.14167993  0.        ]
 [ 0.          0.09983342  0.09983342  0.95056379  0.29404384  0.88206089]
 [ 0.         -0.99500417 -0.99500417  0.09537451  0.02950279 -0.3933314 ]
 [ 1.          0.          0.          0.29552021 -0.95533649  0.25934338]]


In [ ]:
a2 = 1
d1, d4, d6 = 1, 1, 1
#theta = [np.pi/3, np.pi/2, np.pi/4, np.pi/3, 0, np.pi]
theta = [0.1, np.pi/2, 0.3, np.pi/2, 0.5, 0.6]
params = np.array([[0, a2, 0, 0, 0, 0], 
                  [np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0], 
                  [d1, 0, 0, d4, 0, d6], 
                  [theta[0], theta[1], theta[2] + np.pi/2, theta[3], theta[4], theta[5]]])
# params = np.array([[0, a2, 0, 0, 0, 0], 
#                   [np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0], 
#                   [d1, 0, 0, d4, 0, d6], 
#                   [theta[0], theta[1], theta[2], theta[3], theta[4], theta[5]]])

In [ ]:
def homo_trans_mat(a:float, alpha:float, d:float, theta:float) -> np.ndarray:
    c_theta = 0 if (theta == np.pi/2 or theta == -np.pi/2) else np.cos(theta)
    c_alpha = 0 if (alpha == np.pi/2 or alpha == -np.pi/2) else np.cos(alpha)
    
    return np.array([[c_theta, -np.sin(theta)*c_alpha, np.sin(theta)*np.sin(alpha), a*c_theta], 
                     [np.sin(theta), c_theta*c_alpha, -c_theta*np.sin(alpha), a*np.sin(theta)], 
                     [0, np.sin(alpha), c_alpha, d], 
                     [0, 0, 0, 1]])

In [ ]:
# def cal_homo_trans_mat_0_to_n(n:int) -> np.ndarray:
#     '''calculate homogeneous transformation matrix of nth frame w.r.t. base(0th frame)
#     '''
#     T = np.eye(4)
#     for i in range(n):
#         T = T @ homo_trans_mat(params[:, i][0], params[:, i][1], params[:, i][2], params[:, i][3])
#     return T

In [ ]:
def cal_jacobian_mat(params:np.ndarray) -> np.ndarray:
    """
    Compute the Jacobian Matrix
    
    :param params: DH parameters table
    :
    :return J: Jacobian Matrix
    :rtype: numpy.ndarray
    """
    Z_0i = [np.array([0, 0, 1])]  # initialized with Z_00
    Pi = [np.array([0, 0, 0])]    # initialized with P0
    
    T = np.eye(4)
    
    i = 0
    while (i < 6):
        T = T @ homo_trans_mat(params[:, i][0], params[:, i][1], params[:, i][2], params[:, i][3])
        #Z_0i.append(T[:3,:3] @ np.array([0, 0, 1]))
        Z_0i.append(T[:3, 2])     # extracting z-axis
        Pi.append(T[:3, 3])       # extracting position    
    
        i += 1

    J_v = np.array([np.cross(Z_0i[i], Pi[-1] - Pi[i]) for i in range(6)])
    J_w = np.array(Z_0i[:-1])
    
    J = np.vstack([np.column_stack(J_v), np.column_stack(J_w)])

    return J